# Project 5: Working with Pandas and SQL Databases (Movies Dataset)


# Creating an SQLite Database


In [1]:
import sqlite3 as sq3

In [2]:
con= sq3.connect("movies.db")

In [3]:
con

In [4]:
con.execute("SELECT * FROM sqlite_master").fetchall()

[]

In [5]:
con.execute("Select name from sqlite_master WHERE type='table' ORDER BY name").fetchall()

[]

In [6]:
con.close()

# Loading Data from DataFrames into an SQLite Database


In [7]:
import pandas as pd
import json
import sqlite3 as sq3

In [8]:
with open("some_movies.json") as f:
    data=json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'some_movies.json'

In [ ]:
data

In [ ]:
df=pd.json_normalize(data, sep="_")

In [ ]:
df

In [ ]:
movies = df[["id", "title", "revenue", "budget","belongs_to_collection_name", "release_date"]].copy()
movies

In [ ]:
movies.info()

In [ ]:
movies.release_date=pd.to_datetime(df.release_date)

In [ ]:
movies.revenue=df.revenue/1000000
movies.budget=df.budget/1000000

In [ ]:
movies

In [ ]:
votes=df[["id", "vote_count", "vote_average"]].copy()
votes

In [ ]:
genres=pd.json_normalize(data=data, record_path="genres", meta="id", record_prefix="genre_")
genres

In [ ]:
prod=pd.json_normalize(data= data, record_path="production_companies", meta="id", record_prefix="comp_")
prod

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
con

In [ ]:
movies.to_sql("Movies",con, index=False)

In [ ]:
votes.to_sql("Votes", con, index=False)

In [ ]:
genres.to_sql("Genres", con, index=False)

In [ ]:
prod.to_sql("Prod", con, index=False)

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()

In [ ]:
con.execute("Select name FROM sqlite_master where type='table' ORDER BY name").fetchall()

In [ ]:
con.close()

# Loading Data from SQLite Databases into DataFrames

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
con.execute("Select * FROM sqlite_master").fetchall()

In [ ]:
pd.read_sql("Select * FROM sqlite_master",con).info()

In [ ]:
df=pd.read_sql("Select * FROM Movies", con, index_col="id", parse_dates="release_date")
df

In [ ]:
df.info()

In [ ]:
genres=pd.read_sql("SELECT * FROM Genres", con, index_col="id")
genres

In [ ]:
con.close()

#  Some Simple SQL Queries

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM movies", con)

In [ ]:
pd.read_sql("SELECT * \
FROM Movies", con)

In [ ]:
pd.read_sql("SELECT id, revenue, release_date FROM Movies", con)

In [ ]:
pd.read_sql("SELECT sum(revenue) FROM Movies", con)

In [ ]:
con.execute("SELECT sum(revenue) FROM Movies").fetchall()[0][0]

In [ ]:
pd.read_sql("SELECT count(title) FROM Movies", con)

In [ ]:
pd.read_sql("SELECT count(belongs_to_collection_name) FROM Movies", con)

In [ ]:
pd.read_sql("SELECT count(*) FROM Movies", con)

In [ ]:
pd.read_sql("SELECT avg(budget) FROM Movies", con)

In [ ]:
con.close()

# Some more SQL Queries

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE id=597", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE revenue>2000", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE revenue>1500 AND budget<200", con)

In [ ]:
pd.read_sql("SELECT MIN(budget) FROM Movies WHERE revenue>1250", con)

In [ ]:
pd.read_sql("SELECT DISTINCT title FROM Movies", con)

In [ ]:
pd.read_sql("SELECT DISTINCT belongs_to_collection_name FROM Movies", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies ORDER BY budget DESC", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name is NULL", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE belongs_to_collection_name is NOT NULL", con)

In [ ]:
pd.read_sql("SELECT * FROM Movies WHERE title LIKE 'Avengers%'", con)

In [ ]:
con.close()

# Join Queries

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con=sq3.connect("Movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Movies", con)

In [ ]:
pd.read_sql("SELECT * FROM Votes", con)

In [ ]:
pd.read_sql("SELECT * \
FROM Movies \
JOIN Votes \
ON Movies.id=Votes.id", con)

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
FROM Movies \
JOIN Votes \
ON Movies.id=Votes.id", con, index_col="id")

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
FROM Movies \
JOIN Votes \
ON Movies.id=Votes.id \
WHERE Votes.vote_average>8", con, index_col="id")

In [ ]:
pd.read_sql("SELECT Movies.id, Movies.title, Votes.vote_average \
FROM Movies \
JOIN Votes \
ON Movies.id=Votes.id \
WHERE Votes.vote_average>8 \
ORDER BY Movies.budget ASC", con, index_col="id")

In [ ]:
con.close()

# Final Case Study

In [ ]:
import pandas as pd
import sqlite3 as sq3

In [ ]:
con=sq3.connect("movies.db")

In [ ]:
pd.read_sql("SELECT * FROM Prod", con)

In [ ]:
df=pd.read_sql("SELECT Prod.id, Prod.comp_name, Movies.revenue, Movies.title \
              FROM Prod \
              LEFT JOIN Movies \
              ON Prod.id=Movies.id", con)
df

In [ ]:
df.groupby("comp_name").revenue.sum().sort_values(ascending=False)

In [10]:
pd.read_sql("SELECT Prod.comp_name \
FROM Prod \
LEFT JOIN Movies \
ON Prod.id=Movies.id \
WHERE Movies.title= 'Titanic'", con)

ProgrammingError: Cannot operate on a closed database.

In [11]:
df2= pd.read_sql("SELECT Genres.id, Genres.genre_name, Movies.revenue, Movies.title \
FROM Genres \
LEFT JOIN Movies \
ON Genres.id=Movies.id", con)
df2

ProgrammingError: Cannot operate on a closed database.

In [12]:
df2.groupby("genre_name").revenue.sum().sort_values(ascending=False)

NameError: name 'df2' is not defined

In [13]:
pd.read_sql("SELECT Genres.genre_name \
            FROM Genres \
            LEFT JOIN Movies \
            ON Genres.id=Movies.id \
            WHERE Movies.title = 'Frozen II'", con)

ProgrammingError: Cannot operate on a closed database.

In [14]:
con.close()